# imports

In [ ]:
import spacy
import codecs
import pandas as pd
import string
import numpy as np
import re
from collections import Counter
import random
nlp = spacy.load('en_core_web_sm')
#scikitlearn imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report

# HelperFunctions

In [ ]:
class spacy_ops_en(object):
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')
    def __call__(self, some_text):
        doc = self.nlp(some_text)
        return [token.text for token in doc]
    


# Model

In [ ]:
text_clf = Pipeline([   
    ('clf', LogisticRegression(
        random_state = 44,
        solver = 'sag',
        multi_class= 'ovr',
        max_iter=10000
    ))
    
   
])



# Data

In [ ]:
corpus_df = pd.read_csv("train_40k.csv", usecols = ["Score","Text"])
#corpus_df.isna()
print(corpus_df.shape)
corpus_df.head()



In [ ]:
corpus_df = corpus_df[['Text','Score']].dropna()
print(corpus_df.shape)

In [ ]:
corpus_df.Score[corpus_df.Score<=3]=0
corpus_df.Score[corpus_df.Score>=4]=1
ax=corpus_df.Score.value_counts().plot(kind='bar')
fig = ax.get_figure()
fig.savefig("score_boolean.png");

In [ ]:

train_pos_df=corpus_df[corpus_df['Score']==1]
train_neg_df=corpus_df[corpus_df['Score']==0]

print(train_pos_df.shape, train_neg_df.shape)

print(train_neg_df)

# Training DF

In [ ]:
pos_df = train_pos_df[0:8000]
neg_df = train_neg_df[0:8000]
pos_df.reset_index(drop = True, inplace = True)
neg_df.reset_index(drop = True, inplace = True)

train_df=pd.concat([neg_df,pos_df])
# train_df.shape
# train_df.head()
# train_df.tail()

In [ ]:
neg_df

# Test DF

In [ ]:
test_pos_df = train_pos_df[8000:14000]
test_neg_df = train_neg_df[8000:9757]

test_pos_df.reset_index(drop=True, inplace = True)
test_neg_df.reset_index(drop=True, inplace = True)
test_df=pd.concat([neg_df,pos_df])


In [ ]:
test_df.tail()

In [ ]:
#extracting columns for training
X = list(train_df['Text'])
Y = list(train_df['Score'])
X_test = list(test_df['Text'])
Y_test = list(test_df['Score'])


In [ ]:
X_train, X_val, Y_train, Y_val  = train_test_split(X,Y, test_size=0.25, stratify=Y, random_state=45, shuffle = True)
#vectorization
X_train = np.array([nlp(training_text).vector for training_text in X_train])
print("Train VECTORIZED!")
X_val = np.array([nlp(val_text).vector for val_text in X_val])
print("val VECTORIZED!")
X_test = np.array([nlp(test_text).vector for test_text in X_test])
print("Test VECTORIZED!")

In [ ]:
print(X_train[0])
print(Y_train[0])




# Training

In [ ]:
text_clf.fit(X_train,Y_train)

In [ ]:
predictions = text_clf.predict(X_val)


In [ ]:

print(classification_report(y_true=Y_val, y_pred=predictions))